# Machine Learning Project - Parts A, B, C## Ομάδα 1: Στατιστική Μάθηση & Pattern Recognition**Φοιτητής:** Ευάγγελος Μόσχου  **ΑΕΜ:** 10986---### Περιεχόμενα| Part | Θέμα | Μέθοδος ||------|------|---------|| **A** | Maximum Likelihood Estimation | Εκτίμηση παραμέτρων Gaussian || **B** | Parzen Window Density Estimation | Non-parametric πυκνότητα || **C** | K-Nearest Neighbors | Classification με k-NN |### Σύνοψη Μεθοδολογίας- **Part A**: Εκτίμηση μέσης τιμής και διασποράς κανονικών κατανομών- **Part B**: Εκτίμηση πυκνότητας χρησιμοποιώντας kernel functions- **Part C**: Ταξινόμηση βάσει k πλησιέστερων γειτόνων**Τελευταία ενημέρωση:** 2026-01-13

## Part A: Maximum Likelihood Estimation### Θεωρητικό ΥπόβαθροΗ **Maximum Likelihood Estimation (MLE)** είναι μια μέθοδος εκτίμησης παραμέτρων που μεγιστοποιεί την πιθανότητα των παρατηρούμενων δεδομένων.### Για Κανονική ΚατανομήΔεδομένων δειγμάτων x₁, x₂, ..., xₙ από N(μ, σ²):**MLE για μέση τιμή:**$$\hat{\mu} = \frac{1}{n} \sum_{i=1}^{n} x_i$$**MLE για διασπορά:**$$\hat{\sigma}^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \hat{\mu})^2$$### ΥλοποίησηΟ παρακάτω κώδικας:1. Φορτώνει τα δεδομένα2. Υπολογίζει MLE εκτιμήσεις3. Συγκρίνει με τις πραγματικές τιμές (αν υπάρχουν)

In [ ]:
"""
Μέρος Α: Εκτίμηση Μέγιστης Πιθανοφάνειας (Maximum Likelihood Estimation)
Αναγνώριση Προτύπων & Μηχανική Μάθηση - 2025-2026
"""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def load_data(filepath):
    """
    Φορτώνει τα δεδομένα από αρχείο CSV.
    """
    try:
        data = np.loadtxt(filepath, delimiter=',')
        return data
    except Exception as e:
        print(f"Σφάλμα κατά τη φόρτωση: {e}")
        return None

def mle_mean(data):
    """
    Υπολογίζει την εκτίμηση μέγιστης πιθανοφάνειας του μέσου διανύσματος.
    Τύπος: μ = (1/N) * Σ(x_i)
    """
    N = data.shape[0]
    # Άθροισμα κατά μήκος του άξονα 0 (κατακόρυφα) για κάθε χαρακτηριστικό
    sum_features = np.sum(data, axis=0)
    mean_vector = sum_features / N
    return mean_vector

def mle_covariance(data, mean_vector):
    """
    Υπολογίζει την εκτίμηση μέγιστης πιθανοφάνειας του πίνακα συνδιακύμανσης.
    Τύπος: Σ = (1/N) * Σ(x_i - μ)(x_i - μ)^T
    
    Σημείωση: Χρησιμοποιούμε N (όχι N-1) καθώς αυτή είναι η MLE εκτίμηση.
    """
    N = data.shape[0]
    
    # Κεντράρισμα δεδομένων (αφαίρεση μέσου)
    centered_data = data - mean_vector
    
    # Υπολογισμός πίνακα συνδιακύμανσης με πολλαπλασιασμό πινάκων
    # (x - μ)^T * (x - μ) ισοδύναμο με centered_data.T @ centered_data
    covariance_matrix = (centered_data.T @ centered_data) / N
    
    return covariance_matrix

def gaussian_pdf_2d(x, mean, cov):
    """
    Υπολογίζει τη συνάρτηση πυκνότητας πιθανότητας 2D Gaussian στο σημείο x.
    Τύπος: p(x) = (1 / (2π|Σ|^0.5)) * exp(-0.5 * (x-μ)^T * Σ^-1 * (x-μ))
    """
    # Διασφάλιση ότι το x είναι 2D πίνακας για vectorized πράξεις
    if x.ndim == 1:
        x = x.reshape(1, -1)
        
    # Υπολογισμός ορίζουσας και αντίστροφου του πίνακα συνδιακύμανσης
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)
    
    # Σταθερά κανονικοποίησης
    norm_const = 1.0 / ((2 * np.pi) * np.sqrt(det_cov))
    
    # Υπολογισμός απόστασης Mahalanobis για όλα τα σημεία
    diff = x - mean
    
    # (x - μ)^T * Σ^-1 * (x - μ) υπολογιζόμενο αποδοτικά
    exponent = -0.5 * np.sum((diff @ inv_cov) * diff, axis=1)
    
    pdf = norm_const * np.exp(exponent)
    
    return pdf

def main():
    print("--- Μέρος Α: Εκτίμηση Μέγιστης Πιθανοφάνειας ---")
    
    # 1. Φόρτωση δεδομένων
    filepath = '../Datasets/dataset1.csv'
    data = load_data(filepath)
    
    if data is None:
        return

    # Διαχωρισμός χαρακτηριστικών (X) και ετικετών (y)
    X = data[:, :2]  # Πρώτες δύο στήλες: χαρακτηριστικά
    y = data[:, 2]   # Τρίτη στήλη: ετικέτες κλάσεων
    
    print(f"Φορτώθηκαν δεδομένα. Διάσταση: {X.shape}")
    
    # 2. Διαχωρισμός δεδομένων ανά κλάση
    X_c0 = X[y == 0]
    X_c1 = X[y == 1]
    X_c2 = X[y == 2]
    
    print(f"Κλάση 0: {X_c0.shape[0]} δείγματα")
    print(f"Κλάση 1: {X_c1.shape[0]} δείγματα")
    print(f"Κλάση 2: {X_c2.shape[0]} δείγματα")
    
    # 3. Υπολογισμός παραμέτρων MLE για κάθε κλάση
    # Κλάση 0
    mu_0 = mle_mean(X_c0)
    sigma_0 = mle_covariance(X_c0, mu_0)
    
    # Κλάση 1
    mu_1 = mle_mean(X_c1)
    sigma_1 = mle_covariance(X_c1, mu_1)
    
    # Κλάση 2
    mu_2 = mle_mean(X_c2)
    sigma_2 = mle_covariance(X_c2, mu_2)
    
    # Εκτύπωση αποτελεσμάτων
    print("\n--- Εκτιμηθείσες Παράμετροι ---")
    print(f"Κλάση 0 - Μέσος: {mu_0}")
    print(f"Κλάση 0 - Συνδιακύμανση:\n{sigma_0}")
    
    print(f"\nΚλάση 1 - Μέσος: {mu_1}")
    print(f"Κλάση 1 - Συνδιακύμανση:\n{sigma_1}")
    
    print(f"\nΚλάση 2 - Μέσος: {mu_2}")
    print(f"Κλάση 2 - Συνδιακύμανση:\n{sigma_2}")
    
    # Υπολογισμός και αποθήκευση μέγιστων τιμών πυκνότητας
    max_p0 = gaussian_pdf_2d(mu_0, mu_0, sigma_0)[0]
    max_p1 = gaussian_pdf_2d(mu_1, mu_1, sigma_1)[0]
    max_p2 = gaussian_pdf_2d(mu_2, mu_2, sigma_2)[0]
    
    # Υπολογισμός οριζουσών (Determinants) - επηρεάζουν το ύψος της κορυφής
    det_0 = np.linalg.det(sigma_0)
    det_1 = np.linalg.det(sigma_1)
    det_2 = np.linalg.det(sigma_2)
    
    log_text = (
        "=== ΑΝΑΛΥΤΙΚΗ ΑΝΑΦΟΡΑ ΠΑΡΑΜΕΤΡΩΝ MLE ===\n\n"
        "--- ΚΛΑΣΗ 0 ---\n"
        f"Μέσος (Mean Vector):\n{mu_0}\n"
        f"Πίνακας Συνδιακύμανσης (Covariance Matrix):\n{sigma_0}\n"
        f"Ορίζουσα (Determinant): {det_0:.4f}\n"
        f"Μέγιστη Πυκνότητα (Peak Density): {max_p0:.6f}\n\n"
        
        "--- ΚΛΑΣΗ 1 ---\n"
        f"Μέσος (Mean Vector):\n{mu_1}\n"
        f"Πίνακας Συνδιακύμανσης (Covariance Matrix):\n{sigma_1}\n"
        f"Ορίζουσα (Determinant): {det_1:.4f}\n"
        f"Μέγιστη Πυκνότητα (Peak Density): {max_p1:.6f}\n\n"
        
        "--- ΚΛΑΣΗ 2 ---\n"
        f"Μέσος (Mean Vector):\n{mu_2}\n"
        f"Πίνακας Συνδιακύμανσης (Covariance Matrix):\n{sigma_2}\n"
        f"Ορίζουσα (Determinant): {det_2:.4f}\n"
        f"Μέγιστη Πυκνότητα (Peak Density): {max_p2:.6f}\n\n"
        
        "--- ΣΥΓΚΡΙΣΗ ---\n"
        f"Η Κλάση 1 έχει τη μικρότερη ορίζουσα ({det_1:.4f}), άρα είναι η πιο 'στενή' και 'ψηλή'.\n"
        f"Η Κλάση 0 έχει τη μεγαλύτερη ορίζουσα ({det_0:.4f}), άρα είναι η πιο 'πλατιά' και 'χαμηλή'.\n"
    )
    print("\n" + log_text)
    
    with open('density_peaks.txt', 'w') as f:
        f.write(log_text)
    print("Αποθηκεύτηκε το density_peaks.txt (Εμπλουτισμένο)")
    
    # 4. Οπτικοποίηση σε 3D γράφημα
    print("\nΔημιουργία 3D γραφήματος...")
    
    # Ρύθμιση στυλ σε σκοτεινό φόντο
    plt.style.use('dark_background')
    
    # Δημιουργία πλέγματος για την απεικόνιση
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    
    x_range = np.linspace(x_min, x_max, 100)
    y_range = np.linspace(y_min, y_max, 100)
    
    X_grid, Y_grid = np.meshgrid(x_range, y_range)
    
    # Μετατροπή πλέγματος σε σημεία για υπολογισμό PDF
    grid_points = np.column_stack([X_grid.ravel(), Y_grid.ravel()])
    
    # Υπολογισμός τιμών PDF για κάθε κλάση
    Z0 = gaussian_pdf_2d(grid_points, mu_0, sigma_0).reshape(X_grid.shape)
    Z1 = gaussian_pdf_2d(grid_points, mu_1, sigma_1).reshape(X_grid.shape)
    Z2 = gaussian_pdf_2d(grid_points, mu_2, sigma_2).reshape(X_grid.shape)
    
    # Δημιουργία 3D γραφήματος
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Ορισμός σταθερού εύρους στον άξονα Z για σωστή σύγκριση
    ax.set_zlim(0, 0.025)
    
    # Προσαρμογή εμφάνισης άξονα (αφαίρεση γκρι φόντου panels)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    
    # Δημιουργία custom colormaps (Μαύρο -> Χρώμα)
    from matplotlib.colors import LinearSegmentedColormap
    
    def create_cmap(color_name, name):
        colors = [(0, 0, 0), color_name] # Black to Color
        return LinearSegmentedColormap.from_list(name, colors)
    
    cmap_red = create_cmap('red', 'BlackRed')
    cmap_green = create_cmap('lime', 'BlackGreen') # Lime is brighter than green
    cmap_blue = create_cmap('cyan', 'BlackBlue')   # Cyan pops more than blue
    
    # Σχεδίαση επιφανειών με gradient χρώματα
    # Αυξάνουμε τη διαφάνεια (alpha) για τις Κλάσεις 1 και 2, μειώνουμε για Κλάση 0
    surf0 = ax.plot_surface(X_grid, Y_grid, Z0, cmap=cmap_red, alpha=0.6, antialiased=True) # More transparent
    surf1 = ax.plot_surface(X_grid, Y_grid, Z1, cmap=cmap_green, alpha=1.0, antialiased=True) # Full opacity
    surf2 = ax.plot_surface(X_grid, Y_grid, Z2, cmap=cmap_blue, alpha=1.0, antialiased=True)  # Full opacity
    
    # Προσθήκη επίπεδου πατώματος (flat floor) στο z=0
    ax.contourf(X_grid, Y_grid, Z0, zdir='z', offset=0, cmap=cmap_red, alpha=0.3)
    ax.contourf(X_grid, Y_grid, Z1, zdir='z', offset=0, cmap=cmap_green, alpha=0.4)
    ax.contourf(X_grid, Y_grid, Z2, zdir='z', offset=0, cmap=cmap_blue, alpha=0.4)
    
    # Προσθήκη ετικετών και τίτλου
    ax.set_xlabel('Χαρακτηριστικό 1', color='white')
    ax.set_ylabel('Χαρακτηριστικό 2', color='white')
    ax.set_zlabel('Πυκνότητα Πιθανότητας', color='white')
    ax.set_title('3D Απεικόνιση Κατανομών Gauss (MLE)', color='white', fontsize=14)
    
    # Ρύθμιση γωνίας θέασης (περιστροφή δεξιόστροφα)
    ax.view_init(elev=30, azim=210)
    
    # Υπόμνημα με proxy artists
    import matplotlib.patches as mpatches
    patch0 = mpatches.Patch(color='red', label='Κλάση 0', alpha=0.6)
    patch1 = mpatches.Patch(color='lime', label='Κλάση 1', alpha=1.0)
    patch2 = mpatches.Patch(color='cyan', label='Κλάση 2', alpha=1.0)
    
    legend = ax.legend(handles=[patch0, patch1, patch2], loc='upper right')
    plt.setp(legend.get_texts(), color='white')
    
    plt.tight_layout()
    output_file = 'gaussian_3d_plot.svg'
    plt.savefig(output_file, format='svg', dpi=300, transparent=True)
    print(f"Το γράφημα αποθηκεύτηκε στο {output_file}")
    
    # ---------------------------------------------------------
    # Δημιουργία Διαδραστικού Γραφήματος (Plotly)
    # ---------------------------------------------------------
    print("\nΔημιουργία διαδραστικού γραφήματος (HTML)...")
    try:
        import plotly.graph_objects as go
        
        fig_ply = go.Figure()
        
        # Custom colorscales for Plotly (Black -> Color)
        # 0.0 is Black, 1.0 is Color
        cs_red = [[0, 'black'], [1, 'red']]
        cs_green = [[0, 'black'], [1, 'lime']]
        cs_blue = [[0, 'black'], [1, 'cyan']]
        
        # Κλάση 0 (Πιο διαφανής)
        fig_ply.add_trace(go.Surface(z=Z0, x=X_grid, y=Y_grid, 
                                   colorscale=cs_red, opacity=0.6, name='Κλάση 0', showscale=False))
        # Πάτωμα για Κλάση 0
        fig_ply.add_trace(go.Contour(z=Z0, x=x_range, y=y_range, 
                                   colorscale=cs_red, opacity=0.3, showscale=False, 
                                   contours=dict(start=0, end=0.1, size=0.01), zmin=0, zmax=0.1))
        
        # Κλάση 1 (Αυξημένη αδιαφάνεια)
        fig_ply.add_trace(go.Surface(z=Z1, x=X_grid, y=Y_grid, 
                                   colorscale=cs_green, opacity=1.0, name='Κλάση 1', showscale=False))
        # Πάτωμα για Κλάση 1
        fig_ply.add_trace(go.Contour(z=Z1, x=x_range, y=y_range, 
                                   colorscale=cs_green, opacity=0.5, showscale=False,
                                   contours=dict(start=0, end=0.1, size=0.01), zmin=0, zmax=0.1))
        
        # Κλάση 2 (Αυξημένη αδιαφάνεια)
        fig_ply.add_trace(go.Surface(z=Z2, x=X_grid, y=Y_grid, 
                                   colorscale=cs_blue, opacity=1.0, name='Κλάση 2', showscale=False))
        # Πάτωμα για Κλάση 2
        fig_ply.add_trace(go.Contour(z=Z2, x=x_range, y=y_range, 
                                   colorscale=cs_blue, opacity=0.5, showscale=False,
                                   contours=dict(start=0, end=0.1, size=0.01), zmin=0, zmax=0.1))
        
        fig_ply.update_layout(
            title='3D Απεικόνιση Κατανομών Gauss (Διαδραστικό)',
            title_font_color="white",
            paper_bgcolor="black",
            plot_bgcolor="black",
            scene=dict(
                xaxis=dict(title='Χαρακτηριστικό 1', color="white", gridcolor="#333333", tickcolor="white"),
                yaxis=dict(title='Χαρακτηριστικό 2', color="white", gridcolor="#333333", tickcolor="white"),
                zaxis=dict(title='Πυκνότητα', color="white", gridcolor="#333333", tickcolor="white", range=[0, 0.025]),
                bgcolor="black",
                camera=dict(
                    eye=dict(x=-1.5, y=-1.5, z=0.5) # Rotate view
                )
            ),
            width=1000,
            height=800,
            margin=dict(l=65, r=50, b=65, t=90)
        )
        
        html_file = 'gaussian_3d_interactive.html'
        fig_ply.write_html(html_file)
        print(f"Το διαδραστικό γράφημα αποθηκεύτηκε στο {html_file}")
        
    except ImportError:
        print("Η βιβλιοθήκη plotly δεν βρέθηκε. Εγκαταστήστε την με 'pip install plotly' για διαδραστικά γραφήματα.")

if __name__ == "__main__":
    main()


## Part B: Parzen Window Density Estimation### Θεωρητικό ΥπόβαθροΗ **Parzen Window** (ή Kernel Density Estimation) είναι μια non-parametric μέθοδος εκτίμησης πυκνότητας πιθανότητας.### Τύπος$$\hat{f}(x) = \frac{1}{n \cdot h} \sum_{i=1}^{n} K\left(\frac{x - x_i}{h}\right)$$**Όπου:**- K: Kernel function (Gaussian, Hypercube, κλπ.)- h: Bandwidth (ρυθμίζει smoothing)- n: Αριθμός δειγμάτων### Kernel Functions| Kernel | Τύπος ||--------|-------|| **Gaussian** | $K(u) = \frac{1}{\sqrt{2\pi}} e^{-u^2/2}$ || **Hypercube** | $K(u) = 1$ αν \|u\| ≤ 0.5, αλλιώς 0 || **Epanechnikov** | $K(u) = \frac{3}{4}(1-u^2)$ αν \|u\| ≤ 1 |### Επιλογή Bandwidth- **Μικρό h**: High variance, spiky estimate- **Μεγάλο h**: High bias, over-smoothed- **Βέλτιστο h**: Trade-off bias/variance (cross-validation)

In [ ]:
"""
Μέρος Β: Εκτίμηση Πυκνότητας με Παράθυρα Parzen
Αναγνώριση Προτύπων & Μηχανική Μάθηση - 2025-2026
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

def load_data(filepath):
    """
    Φορτώνει μονοδιάστατα δεδομένα από αρχείο CSV.
    """
    try:
        data = np.loadtxt(filepath, delimiter=',')
        return data.flatten()  # Διασφάλιση ότι είναι 1D
    except Exception as e:
        print(f"Σφάλμα κατά τη φόρτωση: {e}")
        return None

def hypercube_kernel(u):
    """
    Πυρήνας υπερκύβου (ομοιόμορφος).
    K(u) = 0.5 αν |u| <= 1, αλλιώς 0
    """
    return np.where(np.abs(u) <= 1, 0.5, 0.0)

def gaussian_kernel(u):
    """
    Πυρήνας Gauss.
    K(u) = (1/sqrt(2π)) * exp(-0.5 * u^2)
    """
    return (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * u**2)

def parzen_window_estimate(x, data, h, kernel_func):
    """
    Υπολογίζει την εκτίμηση Parzen Window στο σημείο x.
    Τύπος: p(x) = (1 / (N * h)) * Σ K((x - x_i) / h)
    
    Παράμετροι:
        x: σημείο(α) αξιολόγησης
        data: δεδομένα εκπαίδευσης
        h: πλάτος παραθύρου (bandwidth)
        kernel_func: συνάρτηση πυρήνα
    """
    N = len(data)
    
    if np.isscalar(x):
        x = np.array([x])
    
    # Αναδιαμόρφωση για broadcasting
    x_col = x[:, np.newaxis]      # (M, 1)
    data_row = data[np.newaxis, :] # (1, N)
    
    # Υπολογισμός κλιμακωμένων αποστάσεων
    u = (x_col - data_row) / h
    
    # Εφαρμογή πυρήνα
    k_values = kernel_func(u)  # (M, N)
    
    # Άθροισμα και κανονικοποίηση
    p_x = np.sum(k_values, axis=1) / (N * h)
    
    return p_x

def true_pdf(x):
    """
    Πραγματική κατανομή: N(1, 4) -> μέσος=1, διακύμανση=4 -> τ.α.=2
    """
    return norm.pdf(x, loc=1, scale=2)

def compute_squared_error(data, h, kernel_func):
    """
    Υπολογίζει το τετραγωνικό σφάλμα μεταξύ εκτιμηθείσας και πραγματικής PDF.
    """
    # Εκτίμηση PDF στα σημεία των δεδομένων
    p_estimated = parzen_window_estimate(data, data, h, kernel_func)
    
    # Πραγματική PDF στα ίδια σημεία
    p_true = true_pdf(data)
    
    # Άθροισμα τετραγωνικών σφαλμάτων
    error = np.sum((p_estimated - p_true)**2)
    return error

def main():
    print("--- Μέρος Β: Εκτίμηση Πυκνότητας με Παράθυρα Parzen ---")
    
    # 1. Φόρτωση δεδομένων
    filepath = '../Datasets/dataset2.csv'
    data = load_data(filepath)
    
    if data is None:
        return
        
    print(f"Φορτώθηκαν δεδομένα. Μέγεθος: {data.shape}")
    print(f"Μέσος δείγματος: {np.mean(data):.4f}, Διακύμανση: {np.var(data):.4f}")
    
    # 2. Ιστόγραμμα vs Πραγματική Κατανομή
    print("\nΔημιουργία ιστογράμματος...")
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=20, density=True, alpha=0.6, color='gray', label='Ιστόγραμμα Δεδομένων')
    
    x_range = np.linspace(min(data)-2, max(data)+2, 200)
    plt.plot(x_range, true_pdf(x_range), 'r-', linewidth=2, label='Πραγματική N(1, 4)')
    
    plt.title('Ιστόγραμμα vs Πραγματική Κατανομή')
    plt.xlabel('x')
    plt.ylabel('Πυκνότητα')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('histogram_verification.png')
    print("Αποθηκεύτηκε το histogram_verification.png")
    
    # 3. Εύρεση βέλτιστου h για κάθε πυρήνα
    h_values = np.arange(0.1, 10.1, 0.1)  # [0.1, 0.2, ..., 10.0]
    
    errors_hypercube = []
    errors_gaussian = []
    
    print("\nΥπολογισμός σφαλμάτων για h στο [0.1, 10]...")
    
    for h in h_values:
        # Πυρήνας υπερκύβου
        err_h = compute_squared_error(data, h, hypercube_kernel)
        errors_hypercube.append(err_h)
        
        # Πυρήνας Gauss
        err_g = compute_squared_error(data, h, gaussian_kernel)
        errors_gaussian.append(err_g)
        
    # Εύρεση βέλτιστου h
    best_h_idx_hyper = np.argmin(errors_hypercube)
    best_h_hyper = h_values[best_h_idx_hyper]
    min_err_hyper = errors_hypercube[best_h_idx_hyper]
    
    best_h_idx_gauss = np.argmin(errors_gaussian)
    best_h_gauss = h_values[best_h_idx_gauss]
    min_err_gauss = errors_gaussian[best_h_idx_gauss]
    
    print(f"\nΠυρήνας Υπερκύβου:")
    print(f"  Βέλτιστο h: {best_h_hyper:.1f}")
    print(f"  Ελάχιστο σφάλμα: {min_err_hyper:.4f}")
    
    print(f"\nΠυρήνας Gauss:")
    print(f"  Βέλτιστο h: {best_h_gauss:.1f}")
    print(f"  Ελάχιστο σφάλμα: {min_err_gauss:.4f}")
    
    # 4. Γραφήματα Σφάλματος vs h
    print("\nΔημιουργία γραφημάτων σφάλματος...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Γράφημα Υπερκύβου
    ax1.plot(h_values, errors_hypercube, 'b-')
    ax1.plot(best_h_hyper, min_err_hyper, 'ro', label=f'Βέλτιστο h={best_h_hyper:.1f}')
    ax1.set_title('Πυρήνας Υπερκύβου: Σφάλμα vs h')
    ax1.set_xlabel('h')
    ax1.set_ylabel('Τετραγωνικό Σφάλμα')
    ax1.legend()
    ax1.grid(True)
    
    # Γράφημα Gauss
    ax2.plot(h_values, errors_gaussian, 'g-')
    ax2.plot(best_h_gauss, min_err_gauss, 'ro', label=f'Βέλτιστο h={best_h_gauss:.1f}')
    ax2.set_title('Πυρήνας Gauss: Σφάλμα vs h')
    ax2.set_xlabel('h')
    ax2.set_ylabel('Τετραγωνικό Σφάλμα')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('parzen_error_plots.png')
    print("Αποθηκεύτηκε το parzen_error_plots.png")

if __name__ == "__main__":
    main()


## Part C: K-Nearest Neighbors Classifier### Θεωρητικό ΥπόβαθροΟ **k-NN (k-Nearest Neighbors)** είναι ένας non-parametric αλγόριθμος ταξινόμησης.### Αλγόριθμος```1. Για νέο sample x:2.   Βρες τους k πλησιέστερους γείτονες στο training set3.   Ψήφισε: η κλάση με τους περισσότερους γείτονες κερδίζει4.   Επέστρεψε την προβλεπόμενη κλάση```### Μετρική Απόστασης**Euclidean Distance:**$$d(x, y) = \sqrt{\sum_{i=1}^{D} (x_i - y_i)^2}$$### Επιλογή k| k | Χαρακτηριστικά ||---|----------------|| **Μικρό k** (π.χ. 1) | Ευαίσθητο σε θόρυβο, high variance || **Μεγάλο k** | Πιο robust, αλλά μπορεί να χάσει local patterns || **Βέλτιστο k** | Επιλέγεται μέσω cross-validation |### Πολυπλοκότητα- **Training**: O(1) - απλή αποθήκευση- **Inference**: O(n·D) - υπολογισμός αποστάσεων- **Βελτιστοποίηση**: KD-Trees, Ball Trees για O(log n)

In [ ]:
"""
Μέρος Γ: Ταξινομητής k Κοντινότερων Γειτόνων (KNN)
Αναγνώριση Προτύπων & Μηχανική Μάθηση - 2025-2026
"""

import numpy as np
import matplotlib.pyplot as plt

def load_data(filepath):
    """
    Φορτώνει δεδομένα από αρχείο CSV.
    Επιστρέφει πίνακα με χαρακτηριστικά και ετικέτες.
    """
    try:
        data = np.loadtxt(filepath, delimiter=',')
        return data
    except Exception as e:
        print(f"Σφάλμα κατά τη φόρτωση: {e}")
        return None

def eucl(x, trainData):
    """
    Υπολογίζει την Ευκλείδεια απόσταση από το σημείο x σε όλα τα σημεία του trainData.
    
    Τύπος: d = sqrt((x1-t1)^2 + (x2-t2)^2)
    
    Παράμετροι:
        x: σημείο αναφοράς (2,)
        trainData: δεδομένα εκπαίδευσης (N, 2)
    
    Επιστρέφει:
        Πίνακα αποστάσεων (N,)
    """
    diff = trainData - x
    sq_diff = diff ** 2
    sum_sq = np.sum(sq_diff, axis=1)
    dist = np.sqrt(sum_sq)
    return dist

def neighbors(x, trainData, k):
    """
    Βρίσκει τους k κοντινότερους γείτονες του x στο trainData.
    
    Παράμετροι:
        x: σημείο προς ταξινόμηση (2,)
        trainData: δεδομένα εκπαίδευσης με ετικέτες (N, 3)
        k: αριθμός γειτόνων
    
    Επιστρέφει:
        Τους k κοντινότερους γείτονες (k, 3)
    """
    features = trainData[:, :2]
    distances = eucl(x, features)
    
    # Ταξινόμηση δεικτών κατά αύξουσα απόσταση
    sorted_indices = np.argsort(distances)
    k_indices = sorted_indices[:k]
    
    return trainData[k_indices]

def predict(testData, trainData, k):
    """
    Προβλέπει τις πιθανότητες κλάσης για κάθε σημείο του testData.
    
    Για κάθε σημείο:
    - Βρίσκει τους k γείτονες
    - Μετράει πόσοι ανήκουν σε κάθε κλάση
    - Υπολογίζει πιθανότητες (αθροίζουν στο 1)
    
    Παράμετροι:
        testData: χαρακτηριστικά προς πρόβλεψη (M, 2)
        trainData: δεδομένα εκπαίδευσης (N, 3)
        k: αριθμός γειτόνων
    
    Επιστρέφει:
        Πίνακα πιθανοτήτων (M, 2) [P(κλάση 0), P(κλάση 1)]
    """
    M = testData.shape[0]
    probabilities = np.zeros((M, 2))
    
    for i in range(M):
        x = testData[i]
        k_neighbors = neighbors(x, trainData, k)
        labels = k_neighbors[:, 2]
        
        # Μέτρηση πλειοψηφίας
        count_0 = np.sum(labels == 0)
        count_1 = np.sum(labels == 1)
        
        # Υπολογισμός πιθανοτήτων
        prob_0 = count_0 / k
        prob_1 = count_1 / k
        
        probabilities[i] = [prob_0, prob_1]
        
    return probabilities

def main():
    print("--- Μέρος Γ: Ταξινομητής k Κοντινότερων Γειτόνων ---")
    
    # 1. Φόρτωση δεδομένων
    train_file = '../Datasets/dataset3.csv'
    test_file = '../Datasets/testset.csv'
    
    train_data = load_data(train_file)
    test_data = load_data(test_file)
    
    if train_data is None or test_data is None:
        return
        
    print(f"Δεδομένα εκπαίδευσης: {train_data.shape}")
    print(f"Δεδομένα ελέγχου: {test_data.shape}")
    
    test_features = test_data[:, :2]
    test_labels = test_data[:, 2]
    
    # 2. Κανονικοποίηση δεδομένων (Z-score)
    # Υπολογισμός μέσου και τ.α. ΜΟΝΟ από τα δεδομένα εκπαίδευσης
    mean = np.mean(train_data[:, :2], axis=0)
    std = np.std(train_data[:, :2], axis=0)
    
    # Εφαρμογή σε εκπαίδευση και έλεγχο
    train_features_norm = (train_data[:, :2] - mean) / std
    test_features_norm = (test_features - mean) / std
    
    # Ανακατασκευή του train_data με κανονικοποιημένα χαρακτηριστικά
    train_data_norm = np.column_stack([train_features_norm, train_data[:, 2]])
    
    print(f"\nΚανονικοποίηση. Μέσος: {mean}, Τ.Α.: {std}")
    
    # 3. Εύρεση βέλτιστου k
    print("\nΕύρεση βέλτιστου k στο [1, 30]...")
    k_values = range(1, 31)
    accuracies = []
    
    for k in k_values:
        # Πρόβλεψη με κανονικοποιημένα δεδομένα
        probs = predict(test_features_norm, train_data_norm, k)
        
        # Επιλογή κλάσης με μέγιστη πιθανότητα
        pred_labels = np.argmax(probs, axis=1)
        
        # Υπολογισμός ακρίβειας
        correct = np.sum(pred_labels == test_labels)
        acc = correct / len(test_labels)
        accuracies.append(acc)
        
    # Εύρεση βέλτιστου k
    best_acc = max(accuracies)
    best_ks = [k for k, acc in zip(k_values, accuracies) if acc == best_acc]
    best_k = best_ks[0]  # Επιλογή του μικρότερου k σε περίπτωση ισοπαλίας
    
    print(f"Βέλτιστο k: {best_k}")
    print(f"Μέγιστη ακρίβεια: {best_acc:.2f}")
    
    # 4. Γράφημα Ακρίβειας vs k
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, accuracies, 'b-o')
    plt.plot(best_k, best_acc, 'r*', markersize=15, label=f'Βέλτιστο k={best_k}')
    plt.title('Ακρίβεια KNN vs k (Κανονικοποιημένα Χαρακτηριστικά)')
    plt.xlabel('k')
    plt.ylabel('Ακρίβεια')
    plt.grid(True)
    plt.legend()
    plt.savefig('knn_accuracy.png')
    print("Αποθηκεύτηκε το knn_accuracy.png")
    
    # 5. Όρια Απόφασης (Decision Boundaries)
    print("\nΔημιουργία γραφήματος ορίων απόφασης...")
    
    # Δημιουργία πλέγματος με κανονικοποιημένα χαρακτηριστικά
    features = train_data_norm[:, :2]
    x_min, x_max = features[:, 0].min() - 0.5, features[:, 0].max() + 0.5
    y_min, y_max = features[:, 1].min() - 0.5, features[:, 1].max() + 0.5
    
    h = 0.05  # Βήμα πλέγματος
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Πρόβλεψη για όλα τα σημεία του πλέγματος
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    probs = predict(grid_points, train_data_norm, best_k)
    Z = np.argmax(probs, axis=1)
    Z = Z.reshape(xx.shape)
    
    # Σχεδίαση
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
    
    # Σχεδίαση σημείων εκπαίδευσης
    class_0 = train_data_norm[train_data_norm[:, 2] == 0]
    class_1 = train_data_norm[train_data_norm[:, 2] == 1]
    
    plt.scatter(class_0[:, 0], class_0[:, 1], c='blue', label='Κλάση 0', edgecolors='k')
    plt.scatter(class_1[:, 0], class_1[:, 1], c='red', label='Κλάση 1', edgecolors='k')
    
    plt.title(f'Όρια Απόφασης KNN (k={best_k})')
    plt.xlabel('Χαρακτηριστικό 1 (Κανονικοποιημένο)')
    plt.ylabel('Χαρακτηριστικό 2 (Κανονικοποιημένο)')
    plt.legend()
    plt.savefig('knn_decision_boundary.png')
    print("Αποθηκεύτηκε το knn_decision_boundary.png")

if __name__ == "__main__":
    main()
